# Product Search Strands Agents with AWS services

## Overview
In this section, we will create a Strands agent for product search and recommendations. The customer will be able to ask the agent for available clothes in the catalogue and any recommendations of clothing for events and seasons.

The steps to complete this notebook are:

1. Install the necessary packages
1. Create S3 Bucket and Bedrock Knowledge Base
1. Create the Strands Agent
1. Add Guardrails to the Agent
1. Access a MCP Server using Agent
1. Test the Agent


## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Native tools used   |retrieve                             |
|Agent Structure     |Single agent architecture                          |
|AWS services used   |Amazon Bedrock (Knowledge Base, Guardrail), Amazon Strands agent, Amazon DynamoDB, Amazon S3   |

</div>


## Architecture

<div style="text-align:center">
    <img src="product-search-architecture.png" width="85% />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with built-in and custom tools
* **Connection with AWS services**: connects with Amazon Bedrock Knoledge Base for information about available clothing
* **Bedrock Model as underlying LLM**: Used Anthropic Claude 4 from Amazon Bedrock as the underlying LLM model

## 1. Install the necessary packages

In [ ]:
!pip install --upgrade -q strands-agents strands-agents-tools boto3 requests-aws4auth

## 2. S3 Bucket and Knowledge Base
The S3 bucket and Bedrock Knowledge Base needed for this workshop have been created in the "prerequisites" lab.

### Import standard libraries

## 3. Create Product Search Agent

### Import libraries including the Strands library
Import the libraries including the Strands and Strands Tools libraries.

In [ ]:
from strands import Agent, tool
from strands_tools import retrieve
from strands.models import BedrockModel

import boto3
from botocore.config import Config as BotocoreConfig

In [ ]:
# Fetch the knowledge base ID of the Product Search Knowledge Base from pre-requisites step
%store -r product_search_kb_id
print(product_search_kb_id)

### Instantiate a simple Agent

In [ ]:
# Create a boto client config with custom settings
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model instance
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name="us-west-2",
    temperature=0.3,
    top_p=0.8,
    boto_client_config=boto_config,
)

# Create the prompt for the agent, specify the instructions and guidelines on the output format
system_prompt = f"""You are an \"AI shopping assistant\", that helps customers discover
  products. You use a Knowledge Base with the id {product_search_kb_id} to search for products based on user preferences 
  and recommends them to the customers. Use confidence threshold of 0.3.
     
  Note: 
    - If you cannot find any matching products, please mention that nothing is available. Do not offer any generic products in the response.
    - Do not ask any questions in the response.
     
  You respond in the below JSON array format - 
  [
     { { "product_id": "PROD-010", 
       "product_name": "Uniqlo Ultra Light Down Jacket", 
       "brand_name": "Uniqlo", 
       "category": "Clothing", 
       "subcategory": "Outerwear", 
       "gender": "Unisex", 
       "price": 69.90, 
       "sale_price": 49.90, 
       "size": ["XS", "S", "M", "L", "XL", "XXL"], 
       "color": ["Black", "Navy", "Red", "Olive", "Grey"], 
       "materials": ["100% Nylon", "Down filling", "Water-repellent coating"], 
       "season": "Fall/Winter" 
       } },
       ...
  ] 
  
"""

# Create the Agent. Pass the "retrieve" tool in the tools list.
product_search_agent = Agent(
    tools=[retrieve],
    model=bedrock_model,
    system_prompt=system_prompt
)

## Test the agent

Let's now test our agent.

In [ ]:
results = product_search_agent("Have you got any baby clothes?")
print(results)

#### Invoking agent with follow up question
Ok, let's now check more clothes and options

In [ ]:
results = product_search_agent("Could you filter the jacket by color and select only the ones in olive?")

In [ ]:
results_jacket = product_search_agent("Do you have any dresses under 100$?")

## 4. Add Guardrails to the Agent

In [ ]:
# Create a Bedrock guardrail

bedrock_client = boto3.client('bedrock')
response = bedrock_client.create_guardrail(
    name='product-specific-restrictions',
    description='Prevents the model from providing recommendations on specific products.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Product restrictions',
                'definition': 'Providing recommendations on product restricted by anycompany and not related to the products available in anycompany.',
                'examples': ['Competitor brands', 'Inappropriate or offensive clothing', 'fake','counterfeit'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'counterfeit'},
            {'text': 'fake'},
            {'text': 'sexy'},
            {'text': 'h&m'},
            {'text': 'nazi'},
            {'text': 'lolita'},
            {'text': 'dupe'}
        ],
        'managedWordListsConfig': [
            {
                'type': 'PROFANITY'
            }
        ]
    },
    blockedInputMessaging='Dear customer, I apologize, but I am not able to provide any recommendations related to your request. Please modify your input and try again.',
    blockedOutputsMessaging='Dear customer, I apologize, but I am not able to provide any recommendations related to your request. Please modify your input and try again.',
)

# Print the response to get the guardrail ID
print("Guardrail ID:", response.get('guardrailId'))
print("Guardrail ARN:", response.get('guardrailArn'))

# Store the guardrail ID for later use

In [ ]:
guardrail_id = response.get('guardrailId')
guardrail_version = "DRAFT"  # Initial version is always 1

### Testing the guardrail directly

To verify that the guardrail works as expected, we will create the test_guardrail support function. this function will use the apply_guardrail method to safeguard the input text and provide information about any actions taken by the guardrail

In [ ]:
# Test function to check if input/output is blocked by guardrail
bedrock_runtime = boto3.client('bedrock-runtime')
def test_guardrail(text, source_type='INPUT'):
      response = bedrock_runtime.apply_guardrail(
          guardrailIdentifier=guardrail_id,
          guardrailVersion=guardrail_version,
          source=source_type,  # can be 'INPUT' or 'OUTPUT'
          content=[{"text": {"text": text}}]
      )

      # New response format uses different fields
      print(f"Action: {response.get('action')}")
      print(f"Action Reason: {response.get('actionReason', 'None')}")

      # Check if content was blocked
      is_blocked = response.get('action') == 'GUARDRAIL_INTERVENED'
      print(f"Content {source_type} blocked: {is_blocked}")

      if is_blocked:
          # Print topic policies that were triggered
          assessments = response.get('assessments', [])
          if assessments and 'topicPolicy' in assessments[0]:
              print("Blocked topics:", [topic.get('name') for topic in
          assessments[0]['topicPolicy'].get('topics', [])
                                       if topic.get('action') == 'BLOCKED'])

          # Print the modified output if available
          if 'outputs' in response and response['outputs']:
              print("Modified content:", response['outputs'][0].get('text', 'None'))

      return response

# Test input that will be blocked. This will be blocked since the input contains
# the diallowed word "lolita".
print("\nTesting input that will be blocked:")
test_guardrail("Can you recommend me a dress lolita style?")

# Test input that will be blocked. This will be blocked since the input contains
# the diallowed word "dupe".
print("\nTesting input that will be blocked:")
test_guardrail("Where can I find a dupe of Chanel jacket?")

# Test some safe input. This will not be blocked.
print("Testing safe input:")
test_guardrail("I am looking for a swimming suit for my holidays in Bali. What can you recommend? I want a 2 pieces")


### Integrating with Strands Agent

Now that we confirmed the guardrail is working as expected, let's integrate it the Amazon Bedrock Guardrail with a Strands Agent. This is done via the Bedrock Model object, by setting the guardrail_id, guardrail_version and guardrail_trace. Once the model object is created you can use it to create your agent. 

In [ ]:
guardrail_id, guardrail_version

In [ ]:
# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    model_id="us.amazon.nova-premier-v1:0",
    region_name="us-west-2",
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version,
    guardrail_trace="enabled",
    temperature=0.3,
    top_p=0.8
)

# Create agent with the guardrail-protected model
product_search_agent_with_guardrail = Agent(
    system_prompt=system_prompt,
    model=bedrock_model,
    tools=[retrieve]
)

### Testing the Strands Agent with Guardrails

Let's test our agent with both safe and risky inputs. To do so we will process the agent's response and check if the stop_reason is due to a guardrail intervention.

In [ ]:
# Helper function to test the agent and check for guardrail interventions
def test_agent_with_guardrail(prompt):
    print(f"\nUser: {prompt}")

    # Get agent response
    response = product_search_agent_with_guardrail(prompt)

    # Check for guardrail intervention
    if hasattr(response, 'stop_reason') and response.stop_reason == "guardrail_intervened":
        print("\n ⚠️ GUARDRAIL INTERVENED!")
        #print(f"Response: {response}")
    else:
        return response

In [ ]:
# Test with a safe question 
test_agent_with_guardrail(
    "What colors are available for pants?"
)

In [ ]:
# Test with a question that asks about hate recommendation. This will get blocked.
test_agent_with_guardrail(
    "Find me a t-shirt with the nazi symbol on it"
)

## 5. Deploy MCP Server

An MCP Server is a lightweight program that exposes specific capabilities through the standardized Model Context Protocol. Host applications (such as chatbots, IDEs, and other AI tools) have MCP clients that maintain 1:1 connections with MCP servers.

In this section, we will deploy a MCP Server that returns reviews of products. We will then use an Agent to get the product reviews from that MCP server.


In [ ]:
# import libraries
import threading
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

Define the Tool.

In [ ]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')

# Create an MCP server
mcp = FastMCP("Reviews server")

# Define a tool to retrieve the reviews
@mcp.tool(description="Retrieve reviews for a specific product ID")
def retrieve_review(product_id: str):
    try:
        dynamodb_table = dynamodb.Table('demo_mcp_product_reviews')
        response = dynamodb_table.query(
            KeyConditionExpression='product_id = :pid',
            ExpressionAttributeValues={
                ':pid': product_id
            }
        )
        if 'Items' in response:
            return response['Items']
        return {"error": "No reviews found for this product"}
    except ClientError as e:
        return {"error": f"Database error: {str(e)}"}
    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}

def main():
    try:
        mcp.run(transport="streamable-http", mount_path="mcp")
    except Exception as e:
        print(f"Server error: {str(e)}")

Start a Thread.

In [ ]:
thread = threading.Thread(target=main)
thread.start()

Now lets create a streamable HTTP MCP client.

In [ ]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")

streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

Finally, create a Strands Agent that uses the MCP client to fetch data from the MCP Server.

In [ ]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()
    agent = Agent(tools=tools)
    response = str(agent("What is the review of PROD-024?"))

## Next Step

Congrats. You have completed Lab 2. Now let's move on to Lab 3. Open `lab 3\inventory-agent-strand` to continue.